# DSCI 100 Group Project: Predicting Disease Diagnoses

## Introduction

The Multiple Disease Prediction dataset uses various health-related parameters (such as glucose and cholesterol levels, insulin levels, and BMI) to check the health of a person - whether they are healthy or afflicted with a specific disease (Diabetes, Thalasse, Anemia, or Thromboc). In this project, we set out to create a classification model to examine to what accuracy we can predict the disease a person with certain parameters can be diagnosed with.

## Exploratory Data Analysis

### Loading libraries

In [1]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)
library(janitor)
library(themis)


Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

### Loading the data

In [2]:
blood_samples <- read_csv("data/blood_samples_dataset.csv")
head(blood_samples)

Rows: 486 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Disease
dbl (24): Glucose, Cholesterol, Hemoglobin, Platelets, White Blood Cells, Re...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,⋯,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein,Disease
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0.001827491,0.03369271,0.11475525,0.9979266,0.56260425,0.86649852,0.57804177,0.91461549,0.02686409,0.03864091,⋯,0.6532299,0.1861045,0.4303978,0.01667792,0.8853520,0.6527334,0.78823514,0.05478811,0.03131344,Thalasse
0.436679025,0.97265269,0.08499819,0.1809086,0.67573568,0.56388877,0.79838214,0.67036126,0.37609238,0.18489008,⋯,0.8335397,0.1530008,0.4585328,0.40184452,0.6359693,0.5744248,0.04702496,0.60798545,0.59412308,Diabetes
0.545697158,0.32481486,0.58446699,0.4757480,0.55859609,0.66100693,0.93405588,0.38178237,0.50034229,0.53182931,⋯,0.6789008,0.2204789,0.8171514,0.69098100,0.1016329,0.8557403,0.55112419,0.41329445,0.07090907,Heart Di
0.172994018,0.05035101,0.73599966,0.7820224,0.06943548,0.08521937,0.03290692,0.46061943,0.78544795,0.49149522,⋯,0.3815002,0.4593956,0.4201545,0.79853707,0.3992360,0.3245997,0.49950360,0.43666181,0.24276586,Diabetes
0.758534051,0.73996775,0.59786776,0.7726835,0.87571974,0.86026493,0.48618863,0.48668607,0.62104804,0.19175598,⋯,0.9933815,0.2723384,0.6635788,0.26522715,0.9188472,0.8049102,0.57111909,0.18836767,0.75084752,Heart Di
0.321872226,0.55464909,0.19565226,0.4272160,0.80775117,0.37696424,0.51952589,0.02808989,0.38253820,0.56820531,⋯,0.7113771,0.8878635,0.3703437,0.60706574,0.9364447,0.1603547,0.10899414,0.59907098,0.56053500,Heart Di


The dataset is already in tidy format where each column is a variable, each row is a single observation, and each cell is a value.
All values in the dataset also have already been centered and scaled to a standard number from 0 to 1. 

The column names are not in the correct format as there are capital leters and spaces. The `Disease` column is also of type character when it should be factor type as there are distinct classes.

We will use clean_names and mutate the `Disease` column to be a factor

In [3]:
blood_samples <- blood_samples |>
                clean_names() |>
                mutate(disease = as_factor(disease))

head(blood_samples)

glucose,cholesterol,hemoglobin,platelets,white_blood_cells,red_blood_cells,hematocrit,mean_corpuscular_volume,mean_corpuscular_hemoglobin,mean_corpuscular_hemoglobin_concentration,⋯,hb_a1c,ldl_cholesterol,hdl_cholesterol,alt,ast,heart_rate,creatinine,troponin,c_reactive_protein,disease
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0.001827491,0.03369271,0.11475525,0.9979266,0.56260425,0.86649852,0.57804177,0.91461549,0.02686409,0.03864091,⋯,0.6532299,0.1861045,0.4303978,0.01667792,0.8853520,0.6527334,0.78823514,0.05478811,0.03131344,Thalasse
0.436679025,0.97265269,0.08499819,0.1809086,0.67573568,0.56388877,0.79838214,0.67036126,0.37609238,0.18489008,⋯,0.8335397,0.1530008,0.4585328,0.40184452,0.6359693,0.5744248,0.04702496,0.60798545,0.59412308,Diabetes
0.545697158,0.32481486,0.58446699,0.4757480,0.55859609,0.66100693,0.93405588,0.38178237,0.50034229,0.53182931,⋯,0.6789008,0.2204789,0.8171514,0.69098100,0.1016329,0.8557403,0.55112419,0.41329445,0.07090907,Heart Di
0.172994018,0.05035101,0.73599966,0.7820224,0.06943548,0.08521937,0.03290692,0.46061943,0.78544795,0.49149522,⋯,0.3815002,0.4593956,0.4201545,0.79853707,0.3992360,0.3245997,0.49950360,0.43666181,0.24276586,Diabetes
0.758534051,0.73996775,0.59786776,0.7726835,0.87571974,0.86026493,0.48618863,0.48668607,0.62104804,0.19175598,⋯,0.9933815,0.2723384,0.6635788,0.26522715,0.9188472,0.8049102,0.57111909,0.18836767,0.75084752,Heart Di
0.321872226,0.55464909,0.19565226,0.4272160,0.80775117,0.37696424,0.51952589,0.02808989,0.38253820,0.56820531,⋯,0.7113771,0.8878635,0.3703437,0.60706574,0.9364447,0.1603547,0.10899414,0.59907098,0.56053500,Heart Di


### Summarizing the training data

Here we will use the distinct function to see how many unique disease classes are in the dataset.

In [4]:
unique_diseases <- select(blood_samples, disease) |>
            distinct()
unique_diseases

disease
<fct>
Thalasse
Diabetes
Heart Di
Anemia
Thromboc
Healthy


Spliting the data into a training subset and a testing subset. 75% of the total data goes to training, 25% to testing.

In [5]:
set.seed(1)

blood_samples_split <- initial_split(blood_samples, prop = 0.75, strata = disease)
training_blood_samples <- training(blood_samples_split)
testing_blood_samples <- testing(blood_samples_split)

How many observations are there for each disease.

In [6]:
obersvation_count <- training_blood_samples |>
            group_by(disease) |>
            summarize(count = n())
obersvation_count

disease,count
<fct>,<int>
Thalasse,35
Diabetes,216
Heart Di,32
Anemia,65
Thromboc,10
Healthy,5


In [7]:
ups_recipe <- recipe(disease ~ ., data = training_blood_samples) |>
    step_upsample(disease, over_ratio = 1, skip = FALSE) |>
    prep()

upsampled_cancer <- bake(ups_recipe, training_blood_samples)


training_blood_samples <- upsampled_cancer |>
            group_by(disease) |>
            summarize(count = n())


### Visualizing the training data

This is a series of histograms comparing the number of a healthy red blood cells a person has.
Anemia is condition where a person doesn't have enough healthy red blood cells. Looking at the normalized graph there is a clear trend that people with Anemia have less healthy red blood cells.

In [8]:
red_blood_histogram <- training_blood_samples |> 
                        ggplot(aes(x=red_blood_cells, fill = disease)) +
                        geom_histogram(bins = 8) +
                        facet_grid(rows = vars(disease))
red_blood_histogram

ERROR while rich displaying an object: Error in `geom_histogram()`:
! Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error:
! object 'red_blood_cells' not found

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = 

## Methods

Because the parameters required to predict each disease are not necessarily the same, we will need to identify which parameters dictate the diagnoses of each disease. One way this could be done is by creating histograms to visualize the distribution of each parameter per each disease, and, by comparing these with the parameter histograms of the other diseases, determine if there are patterns for parameters amongst each disease.

Then, once we have identified which parameters dictate for each disease, we can build our classification model(s) to identify which disease a person with certain parameters could be diagnosed with.

In [9]:
disease_recipe <- recipe(disease ~ ., data = training_blood_samples)

disease_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")


k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 10))

disease_vfold <- vfold_cv(training_blood_samples, v = 5, strata = disease)


tune_results <- workflow() |>
  add_recipe(disease_recipe) |>
  add_model(disease_spec) |>
  tune_grid(resamples = disease_vfold, grid = k_vals) |>
  collect_metrics()

tune_results

ERROR: [1m[33mError[39m in `purrr::map()`:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error in `rsplit()`:[22m
[33m![39m At least one row should be selected for the analysis set.


In [ ]:
accuracies <- tune_results |>
  filter(.metric == "accuracy")

accuracies

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") +
  theme(text = element_text(size = 12))

accuracy_vs_k

## Expected Outcomes

Through our data analysis, we anticipiate there will be relationships between health parameters and specific diseases. We hypothesize that there will be patterns where elevated or reduced levels of a particular health parameter will predict the presence of a disease. Although for the purposes of this course, our model will be fairly naive, to create an accurate model that can achieve such predictions would have an impact on healthcare. Such predictions can allow these at-risk individuals to seek the support they need. Therefore, it also raises a number of potential future questions:
- Are there additional health parameters that should be measured for training the model?
- Are there ethical concerns regarding the use of using classification models to predict health risks?
- How would an individual input such information into the model to receive a diagnosis?
- We have only learned about k-nearest neighbour classifiers in this class, is there a different classifier that would perform better?